# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

* Задание 1.

*Обязательная часть*

Будем парсить страницу со свежеми новостям на habr.com/ru/all/.

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

KEYWORDS = ['python', 'парсинг']

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).

В итоге должен формироваться датафрейм вида: <дата> - <заголовок> - <ссылка>

Дополнительная часть (необязательная)
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>

In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime
import time
import re
import json

In [2]:
KEYWORDS = ['python', 'парсинг']

In [3]:
URL = 'https://habr.com/ru/search/'
params = {
    'q':  KEYWORDS
}

links = []
titles = []
dates = []
res = requests.get(URL, params)
soup = BeautifulSoup(res.text, 'html.parser')
posts = soup.find_all('article', class_='post', lang = 'ru')
for post in posts:
    link = post.find('a', class_='post__title_link').get('href')
    links.append(link)
    title = post.find('h2', class_='post__title').text.strip('\n')
    titles.append(title)
    date = str(post.find('span', class_='post__time'))
    # или date = (' ').join(re.findall(r'\w+', date)[3:6])
    date = str(re.findall(r'(?<=>)(.+(?=\sв))', date)).strip('[]').strip("' '")
    dates.append(date)
dict_ = {'date':dates, 'title':titles, 'link':links}
dt = pd.DataFrame(dict_)
# pd.set_option('display.max_colwidth', None)  для вывода полных строк
dt

,date,title,link
0,3 апреля 2019,"Правда про парсинг сайтов, или «все интернет-м...",https://habr.com/ru/post/446488/
1,17 октября 2017,Парсинг сайтов: как с ...,https://habr.com/ru/post/340302/
2,7 мая 2019,Парсинг сайтов — а это вообще легально в России?,https://habr.com/ru/post/450834/
3,1 октября 2020,Как выбрать решение для парсинга сайтов: класс...,https://habr.com/ru/post/521646/
4,1 декабря 2016,Пример использования Product API от Fetchee дл...,https://habr.com/ru/company/fetchee/blog/316558/
5,3 марта 2016,Теория и практика парсинга исходников с помощь...,https://habr.com/ru/company/pt/blog/210772/
6,18 июня 2018,Парсинг и работа с Codable в Swift 4,https://habr.com/ru/post/414221/
7,28 августа 2019,Парсинг и анализ семантики для SEO: 5 бесплатн...,https://habr.com/ru/company/click/blog/465277/
8,14 июня 2011,Парсинг на Pуthon. Как собрать архив Голубятен,https://habr.com/ru/post/121815/
9,18 марта 2015,Продвинутый парсинг веб-сайтов с Mechanize,https://habr.com/ru/post/253439/


In [4]:
URL2 = 'https://habr.com/ru/search/'

def parsing (URL, query):
    params = {
        'q':  query
    }
    dt = pd.DataFrame()
    links = []
    titles = []
    dates = []
    texts = []
    res = requests.get(URL, params)
    soup = BeautifulSoup(res.text, 'html.parser')
    posts = soup.find_all('article', class_= 'post', lang = 'ru')
    for post in posts:
        link = post.find('a', class_= 'post__title_link').get('href')
        links.append(link)
        soup = BeautifulSoup(requests.get(link).text, 'html.parser')
        title = soup.find('h1', class_= 'post__title').text.strip('\n')
        titles.append(title)
        date = pd.to_datetime(soup.find('span', class_= 'post__time').get('data-time_published'), dayfirst=True).date()
        dates.append(date)
        text = soup.find('div', class_='post__text').text
        texts.append(text)
        row = {'date':dates, 'title':titles, 'link':links, 'text':texts}
        dt = pd.concat([dt, pd.DataFrame(row)])
        # pd.set_option('display.max_colwidth', None) для вывода полных строк
        # pd.option_context('display.max_columns', None) для вывода всех колонок без переноса
        # pd.options.display.expand_frame_repr = False  для вывода всех колонок без переноса
    return dt

parsing (URL2, KEYWORDS)

,date,title,link,text
0,2019-04-03,"Правда про парсинг сайтов, или «все интернет-м...",https://habr.com/ru/post/446488/,В этой статье я постараюсь наиболее просто рас...
0,2019-04-03,"Правда про парсинг сайтов, или «все интернет-м...",https://habr.com/ru/post/446488/,В этой статье я постараюсь наиболее просто рас...
1,2017-10-17,Парсинг сайтов: как с точки зрения закона выгл...,https://habr.com/ru/post/340302/,\r\nПопробуем рассмотреть один из лучших спосо...
0,2019-04-03,"Правда про парсинг сайтов, или «все интернет-м...",https://habr.com/ru/post/446488/,В этой статье я постараюсь наиболее просто рас...
1,2017-10-17,Парсинг сайтов: как с точки зрения закона выгл...,https://habr.com/ru/post/340302/,\r\nПопробуем рассмотреть один из лучших спосо...
...,...,...,...,...
15,2015-07-19,Использование morph.io для веб-парсинга,https://habr.com/ru/post/262991/,"Если вы читали предыдущие две статьи, Веб-парс..."
16,2013-02-14,Парсинг сайтов-магазинов. Личный опыт и немног...,https://habr.com/ru/post/169409/,Разделим парсинг (скраппинг) сайтов на две под...
17,2009-05-25,Еще о парсинге на Prolog'е,https://habr.com/ru/post/60430/,"Вот тут наткнулся на, в общем-то, простую зада..."
18,2011-09-02,Grab — python библиотека для парсинга сайтов,https://habr.com/ru/post/127584/,"Лет пять-шесть назад, когда я ещё программиров..."


* Задание 2.

Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса Avast Hack Ckeck. Список email-ов задаем переменной в начале кода:
EMAIL = [xxx@x.ru, yyy@y.com]

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>

Подсказка: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.
Документация к API VK: https://vk.com/dev/methods , вам поможет метод wall.get

GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>

In [13]:
EMAIL = ["flovizi@mail.ru", "flovizi@list.ru"]

In [124]:
# ни при каких условиях мне не удается извлечь инфо (( 
URL="https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data"
data={
    "emailAddresses":EMAIL
}
headers={
    "Vaar-Header-App-Product":"hackcheck-web-avast",
    "Vaar-Version":"0"
}
response=requests.post(URL, json = data, headers=headers)
response.text
response.json()
r = pd.DataFrame(response.json())
r

,breaches,data,summary
11,"{'breachId': 11, 'site': 'badoo.com', 'records...",NaN,NaN
badoo.com,NaN,"{'flovizi@list.ru': [{'breachId': 11, 'usernam...",NaN
flovizi@list.ru,NaN,NaN,{'breaches': [11]}
flovizi@mail.ru,NaN,NaN,{'breaches': []}


In [97]:
TOKEN = pd.read_excel(r'C:\Users\Urmas\Desktop\Jupyter\test\Parsing API\token.xlsx')
GROUP = 'netology'
VERSION = '5.126'

In [102]:
wall = 'https://api.vk.com/method/wall.get?'
params = {
    'access_token': TOKEN,
    'v': VERSION,
    'domain': GROUP,
    'count': 50,
    'extended': 1
}

In [133]:
res = requests.get(wall, params)
res
res.text
res.json()
re = pd.DataFrame(res.json()['response']['items'])
re['data'] = re['date'].apply(lambda x: datetime.fromtimestamp(x))
re[['data', 'text']].head()

,data,text
0,2020-12-07 13:05:00,Наш предновогодний digital-марафон продолжаетс...
1,2020-12-08 10:42:00,"Кто-то только начал заниматься копирайтингом, ..."
2,2020-12-07 17:49:00,Новый год на носу — самое время подводить итог...
3,2020-12-07 10:19:00,📝 Записывайтесь на бесплатный курс «Диджитал-с...
4,2020-12-06 11:38:00,"Когда дедлайн по проекту уже близко, то менедж..."
